In [ ]:
# Required
import matplotlib.pyplot as plt
import numpy

# Helpers
from dataclasses import dataclass
import pytest

In [ ]:
@dataclass(frozen=True)
class Population(object):

    generation: int
    value: float
    growth_rate: float
    init: float
    history: list[float]

    @classmethod
    def new(cls, r: float, x0: float) -> 'Population':
        return Population(
            growth_rate = r,
            history = [x0],
            generation = 1,
            init = x0,
            value = x0,
        )
    
    def next_generation(self) -> 'Population':
        r: float = self.growth_rate
        x: float = self.value
        x1 = r * x * (1 - x)

        history = self.history + [x1]
        generation = len(history)
        
        return Population(
            growth_rate = self.growth_rate,
            history = history,
            generation = generation,
            init = history[0],
            value = history[ generation - 1 ],
        )

    def next_nth_generations(self, n: int) -> 'Population':
        N = n + self.generation
        return self.until_nth_generation(N)
    
    def until_nth_generation(self, N: int) -> 'Population':
        while self.generation < N:
            self = self.next_generation()
        return self

    # Helpers

    def print(self):
        print(self.__dict__)

    def validate(self, dict): 
        self.print()
        assert population.growth_rate == dict['growth_rate']
        assert population.init == dict['init']
        assert population.value == dict['value']
        assert population.history == dict['history']
        assert population.generation == dict['generation']

In [ ]:
population = Population.new(r=1.0, x0 = 0.5)
population.validate({'generation': 1, 'value': 0.5, 'growth_rate': 1.0, 'init': 0.5, 'history': [0.5]})

population = population.next_generation()
population.validate({'generation': 2, 'value': 0.25, 'growth_rate': 1.0, 'init': 0.5, 'history': [0.5, 0.25]})

population = population.until_nth_generation(15)
population.validate({'generation': 15, 'value': 0.05357062532685648, 'growth_rate': 1.0, 'init': 0.5, 'history': [0.5, 0.25, 0.1875, 0.15234375, 0.1291351318359375, 0.11245924956165254, 0.09981216674968249, 0.08984969811841606, 0.08177672986644556, 0.07508929631879595, 0.06945089389714401, 0.06462746723403166, 0.06045075771294583, 0.05679646360487655, 0.05357062532685648]})

population = population.next_nth_generations(10)
population.validate({'generation': 25, 'value': 0.03433841067421475, 'growth_rate': 1.0, 'init': 0.5, 'history': [0.5, 0.25, 0.1875, 0.15234375, 0.1291351318359375, 0.11245924956165254, 0.09981216674968249, 0.08984969811841606, 0.08177672986644556, 0.07508929631879595, 0.06945089389714401, 0.06462746723403166, 0.06045075771294583, 0.05679646360487655, 0.05357062532685648, 0.05070081342894604, 0.04813024094658925, 0.04581372085301251, 0.04371482383461476, 0.041803838011723354, 0.04005627713921295, 0.03845177180095951, 0.036973233046326444, 0.035606213084428476, 0.03433841067421475]})

### Plot functions

In [ ]:
def plot_population_over_time(population: Population):
    x = range(population.generation)
    y = population.history
        
    plt.plot(x, y)

def plot_convergence_values(list_population_histories: list[dict]):
    R = []
    C = []

    hists = list_population_histories

    for i in range(0, len(hists)):
        r = hists[i]['growth_rate']
        conv = hists[i]['convergence_value']
        R = R + [r]
        C = C + [conv]

    plt.scatter(R, C)

In [ ]:
plot_population_over_time(population)

### Convergence value

In [ ]:
def convergence_value(series: list[float]) -> float:
    derivative_first = list( numpy.gradient( series ))
    derivative_second = list( numpy.gradient( series ))

    n: int = len(series) - 1
    value_nth = series[n]
    angle_nth = derivative_first[n]
    curvature_nth = derivative_second[n]

    if is_zero(angle_nth) and is_zero(curvature_nth):
        return value_nth
    else: 
        return None

def is_zero(value) -> bool:
    limit = 5e-7
    return limit > abs(value)
    

In [ ]:
r = 1.0
x = 0.5
n = 150
X = population_nth_generation(r, [x], n)

convergence_value(X)

## Find convergence value starting from x0

In [ ]:
def find_convergence(growth_rate: float, population_history: list, try_until: int = 5000):
    r: float = growth_rate
    X: list = population_history

    if len(X) < 10:
        X = population_nth_generation(r, [x], 10)

    convergence: float = None

    while convergence is None and len(X) < try_until:
        X = population_nth_generation(r, X, 1)
        convergence = convergence_value(X)

    # TODO - create struct
    return {
        'growth_rate': growth_rate,
        'initial_value': population_history[0],
        'convergence_value': convergence,
        'generation': len(X) - 1,
        'series': X,
    }

In [ ]:
r = 1.0
x = 0.5

convergence = find_convergence(r, [x], 5000)

print(convergence['convergence_value'])
print(convergence['generation'])
plot_population_over_time(convergence['series'])

### Build datatable r vs x vs n

In [ ]:
x = 0.5
r0 = 1.0
rN = 4.0
r_step = 0.1

results = []

for r in numpy.arange(r0, rN, r_step):
    result = find_convergence(r, [x])
    results = results + [result]

In [ ]:
plot_convergence_values(results)